In [3]:
!pip install gradio

  Obtaining dependency information for gradio from https://files.pythonhosted.org/packages/05/f2/360ca9546cffa45fee1df56864fdc2b6955de622e98435539490cd882a96/gradio-3.47.1-py3-none-any.whl.metadata
  Obtaining dependency information for aiofiles<24.0,>=22.0 from https://files.pythonhosted.org/packages/c5/19/5af6804c4cc0fed83f47bff6e413a98a36618e7d40185cd36e69737f3b0e/aiofiles-23.2.1-py3-none-any.whl.metadata
  Obtaining dependency information for altair<6.0,>=4.2.0 from https://files.pythonhosted.org/packages/17/16/b12fca347ff9d062e3c44ad9641d2ec50364570a059f3078ada3a5119d7a/altair-5.1.2-py3-none-any.whl.metadata
  Obtaining dependency information for fastapi from https://files.pythonhosted.org/packages/4d/d2/3ad038a2365fefbac19d9a046cab7ce45f4c7bfa81d877cbece9707de9ce/fastapi-0.103.2-py3-none-any.whl.metadata
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for gradio-client==0.6.0 from https://files.pythonhosted.org/packages/9f/dd/db8dcc8521aa475a18a562929d

In [74]:
import re
import random
import gradio as gr
import requests
import json

# Global
controller_url = "ip-172-31-73-215.ap-southeast-2.compute.internal" + ":8000"
l_mapping = {
    "Bash":"shell", 
    "Python":"python", 
    "Java":"javascript",
    "JavaScript":"javascript", 
    "R":"r"
}
models_list = {
    "Claude-v2": {
        "model_family": "bedrock",
        "model_name": "anthropic.claude-v2"
    }
}
css = """
#red {background-color: #FA9F9D}
#amber {background-color: #FFD966}
"""



def compare(exp_out, out):
    """
    Compare expected output with output
    """
    if exp_out == out:
        return 
    elif "Error" or "error" in out : 
        return gr.Textbox(value=out,label="Error", elem_id="red")
    else:
        return gr.Textbox(value=out,label="Expected Output does not match Output", elem_id="amber")
    
def execute_fn(state, history, script, exp_out, model, language):
    data = json.dumps(
        {
            "script": script,
            "model_family": models_list[model]["model_family"], 
            "model_name": models_list[model]["model_name"], 
            "language": language,
            "expected_output": exp_out,
            "conv_id": state
        }
    )
    response = requests.post(
            "http://" + controller_url + "/execute",
            data=data
        ).json()
    if response["error"]:
        out = gr.Textbox(value=response["output"],label="Error", elem_id="red")
        history[-1][1] = response["generated_text"]
    else:
        if response["output"] != exp_out:
            out = gr.Textbox(value=response["output"],label="Expected Output does not match Output", elem_id="amber")
        else:
            out = gr.Textbox(value=response["output"],label="Output")
    return [history, out] 


def can_exec(script):
    if script != "":
        return gr.Button(value="Approve and Execute", interactive=True)
    else:
        return gr.Button(value="Approve and Execute", interactive=False)

    
def Dropdown_lang(dropdown, script):
    """
    Code Languages
    approved language: [('python', 'markdown', 'json', 'html', 'css', 'javascript', 'typescript', 'yaml', 'dockerfile', 'shell', 'r')]
    """
    return [gr.Code(value=script, language=l_mapping[dropdown], interactive=False)]

def clear_fn():
    return [""] * 3 + [gr.Textbox(value="",label="Output", elem_id="--primary-50")]

def generate_response(state, message, history, model, language): # this function will be replaced with the LLM response
    """
    Sample Response - to be deleted
    """
    data = json.dumps(
        {
            "prompt": message,
            "model_family": models_list[model]["model_family"], 
            "model_name": models_list[model]["model_name"], 
            "language": language,
            "conv_id": state
        }
    )
    response = requests.post(
            "http://" + controller_url + "/generate",
            data=data
        ).json()
    state = response["conv_id"]
    history += [[message, response["generated_text"]]]
    return [state, history, response["script"], response["expected_output"]]

def web_ui():
    with gr.Blocks(css=css, theme=gr.themes.Base(neutral_hue="slate")) as webUI:
        state = gr.State(value="")
        gr.Markdown(
            """
            # Welcome to SynthCodeX
            Use this tool to generate and test code using LLMs
            """)
        with gr.Row():
            language = gr.Dropdown(languages,label='Langauge Selection', value=languages[0]) # Langauge Selection
            model = gr.Dropdown(models_list.keys(),label="Model Selection", value=list(models_list.keys())[0]) # Model Selection
        # bot = gr.Chatbot(render=False)
        with gr.Row(equal_height=True):
            chat = gr.ChatInterface(generate_fake_rsps, undo_btn=None, retry_btn=None) #, additional_inputs=[state])
            # dropdown.input(fn=reset, inputs=dropdown, outputs=[bot, chat.chatbot_state]) # Reset the chatbot to clear history automatically
            with gr.Column():
                script = gr.Code(value="",label="Script", language = l_mapping[languages[0]], interactive=False)
                exp_out = gr.Textbox(value="",label="Expected Output")
                out = gr.Textbox(value="",label="Output")
                auto_run = gr.Checkbox(label='Auto-approve and Execute')
                timeout = gr.Number(label="Timeout", precision=0, minimum=0, maximum=60)
                execute = gr.Button(value="Approve and Execute", interactive=False)
            
        language.change(Dropdown_lang, [language, script], [script, out])
        out.change(compare, [exp_out, out], out)
        script.change(can_exec, script, execute)
        chat.submit_btn.click(generate_response, [state, chat.textbox, chat.chatbot, model, language], [state, chat.chatbot, script, exp_out])
        chat.textbox.submit(generate_response, [state, chat.textbox, chat.chatbot, model, language], [state, chat.chatbot, script, exp_out])
        chat.clear_btn.click(clear_fn, None, [state, script, exp_out, out])
        execute.click(execute_fn, [state, chat.chatbot, script, exp_out, model, language], [chat.chatbot, out])
    return webUI

def get_languages_list(url):
    return list(json.loads(requests.get("http://" + url + "/list_languages").json()).keys())
    
if __name__ == "__main__":    
    languages = get_languages_list(controller_url)
    UI = web_ui()
    UI.launch(debug=True)

    # create events, when chaning model/language selection, it will clear the chat history, scirpt, expected output and output


Running on local URL:  http://127.0.0.1:7862
Sagemaker notebooks may require sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Running on public URL: https://0ecbd7a0cef4992093.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.


KeyboardInterrupt: 

# Reset Sample

In [ ]:
import gradio as gr

def converse(x, y, z):
    return z

def reset(z):
    return [], []

# Gradio app
with gr.Blocks() as demo:
    bot = gr.Chatbot(render=False)
    dropdown = gr.Dropdown(["Character 1", "Character 2", "Character 3"], label="Characters", info="Select the character that you'd like to speak to", value="Character 1")
    chat = gr.ChatInterface(
        fn=converse,
        chatbot=bot,
        additional_inputs=dropdown
    )
    dropdown.input(fn=reset, inputs=dropdown, outputs=[bot, chat.chatbot_state])

demo.queue()
demo.launch()

In [44]:
"http://" + controller_url + "/generate"

'http://ip-172-31-73-215.ap-southeast-2.compute.internal:8000/generate'